In [29]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [30]:

from pymongo.mongo_client import MongoClient

MONGODB_ATLAS_CLUSTER_URI = os.getenv("MONGODB_ATLAS_CLUSTER_URI")

# Create a new client and connect to the server
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = ""
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [24]:
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient

# initialize MongoDB python client
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

DB_NAME = "langchain_test_db"
COLLECTION_NAME = "langchain_test_vectorstores"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "langchain-test-index-vectorstores"

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

vector_store = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    embedding=embeddings,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    relevance_score_fn="cosine",
)

# Create vector search index on the collection
# Since we are using the default OpenAI embedding model (ada-v2) we need to specify the dimensions as 1536
vector_store.create_vector_search_index(dimensions=3072)

In [31]:
MONGODB_COLLECTION

Collection(Database(MongoClient(host=['ac-hwqbhhk-shard-00-00.gwdg650.mongodb.net:27017', 'ac-hwqbhhk-shard-00-02.gwdg650.mongodb.net:27017', 'ac-hwqbhhk-shard-00-01.gwdg650.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster0', authsource='admin', replicaset='atlas-vbgwkz-shard-0', tls=True), 'langchain_test_db'), 'langchain_test_vectorstores')

In [25]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['d5a8ec52-8b80-4714-9007-260936741519',
 'b81a0632-ec08-42b2-8381-eeb4f6b9a504',
 'a28fe0df-28c9-43cf-9795-a261c2747a6e',
 'c74139f6-0bb9-41f1-89da-3d83c41255ce',
 '82a90ccd-aa46-494d-833f-b37e1eac613d',
 '61be2d6a-52c0-45b3-9bc9-73c37cad3eac',
 '80ec8fad-f340-4abe-b5f9-bdcc9abedc43',
 '831f8909-ddde-4abc-a224-626a518cf6e8',
 '628ac9ac-9acb-4534-8c7d-ffcfe59ca20f',
 'a38a840a-0d71-43b2-bd52-0d06aa04a94e']

In [26]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.2},
)
retriever.invoke("Stealing from the bank is a crime")

[Document(id='c74139f6-0bb9-41f1-89da-3d83c41255ce', metadata={'_id': 'c74139f6-0bb9-41f1-89da-3d83c41255ce', 'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

In [10]:
from pymongo.mongo_client import MongoClient
from dotenv import load_dotenv
import os
load_dotenv()
MONGODB_ATLAS_CLUSTER_URI = os.getenv("MONGODB_ATLAS_CLUSTER_URI")
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)
db = "langchain_test_db"
collection = "langchain_test_vectorstores"
index_ = "langchain-test-index-vectorstores"
MONGODB_COLLECTION = client[db][collection]

existing_indexes = MONGODB_COLLECTION.list_indexes()
index_exists = any(index_ in idx for idx in existing_indexes)
index_exists

False

In [9]:
existing_search_indexes = list(MONGODB_COLLECTION.aggregate([{"$listSearchIndexes": {}}]))
index_exists = any(index_ == i['name'] for i in existing_search_indexes)
index_exists

True